# Logging in R: Tutorial

In [1]:
library(logging)

In [2]:
basicConfig()

## The basics

In [3]:
ls(getLogger())

[1] "addHandler"    "handlers"      "level"         "msg_composer" 
[5] "name"          "removeHandler"

In [4]:
getLogger()[['level']]

INFO 
  20

In [5]:
getLogger()[['handlers']]

$basic.stdout
<environment: 0x7ff39b47e3c8>


In [6]:
loginfo('does it work?')

2019-07-21 01:26:09 INFO::does it work?


In [7]:
logwarn('my %s is %d', 'name', 5)

2019-07-21 01:26:09 WARNING::my name is 5


In [8]:
logdebug('I am a silent child')

We add an other handler to the console, without specifying its name.  It gets one automatically from the name of the function.  You can add and remove handlers using their names.  You can also refer to them by function, if that is the way you define it.

In [9]:
addHandler(writeToConsole)

In [10]:
names(getLogger()[['handlers']])

[1] "basic.stdout"   "writeToConsole"

In [11]:
loginfo('test')

2019-07-21 01:26:10 INFO::test
2019-07-21 01:26:10 INFO::test


In [12]:
logwarn('test')

2019-07-21 01:26:10 WARNING::test
2019-07-21 01:26:10 WARNING::test


In [13]:
removeHandler('writeToConsole')

In [14]:
logwarn('test')

2019-07-21 01:26:10 WARNING::test


Handlers have a level associated to them.  Any logging record passing through a handler and having a severity lower than the level of the handler is ignored.  You can alter the level of a handler.  This is what we do in the following lines: we alter the level of the default console handler 'basic.stdout' to 30 (**WARN**).

In [15]:
addHandler(writeToConsole)

In [16]:
setLevel(30, getHandler('basic.stdout'))

In [17]:
logwarn('test')

2019-07-21 01:26:10 WARNING::test
2019-07-21 01:26:10 WARNING::test


In [18]:
loginfo('test')

2019-07-21 01:26:10 INFO::test


In [19]:
getHandler('basic.stdout')[['level']]

WARNING 
     30

In [20]:
getHandler('writeToConsole')[['level']]

NOTSET 
     0

## Hierarchical loggers

In the previous section we have worked *implicitly* with one logger, the root logger.  We can refer to it explicitly by specifying the 'logger' parameter in our function calls.  The name of the root logger is the empty string.  This also explains that "::" in the messages sent to the console, between the first and the second ":" there's the name of the logger that is associated to the log record shown.

In [21]:
with(getLogger(logger=''), names(handlers))

[1] "basic.stdout"   "writeToConsole"

In [22]:
with(getLogger('libro'), names(handlers))

NULL

when issuing a logging record, you can specify to which logger you want to send it.  **Loggers are created when first needed**, so we can just assume all loggers we need also exist.  The logger will offer it to all its attached handlers and then pass it to its parent logger.  Loggers are organized hierarchically, in a way that is similar to the way directories are organized.

Just as directories contain files, loggers contain handlers and their name is, within the logger, unique.  Also similar to directories, all loggers have one parent, except the root logger that has none.  **The name of the logger specifies the location of the logger in this hierarchy.**

Let's start from scratch, either a brand new R session or by resetting the logging system.

In [23]:
logReset()

In [24]:
addHandler(writeToConsole, logger='libro.romanzo')

In [25]:
loginfo('Ma cos\'è questo amore?', logger='libro.romanzo.companile')

2019-07-21 01:26:10 INFO:libro.romanzo.companile:Ma cos'è questo amore?


In [26]:
loginfo('Se la luna mi prta fortuna', logger='libro.romanzo.companile')

2019-07-21 01:26:10 INFO:libro.romanzo.companile:Se la luna mi prta fortuna


In [27]:
loginfo('Giovanotti, non esageriamo!', logger='libro.romanzo.companile')

2019-07-21 01:26:10 INFO:libro.romanzo.companile:Giovanotti, non esageriamo!


In [28]:
loginfo('memories of a survivor', logger='libro.romanzo.lessing')

2019-07-21 01:26:10 INFO:libro.romanzo.lessing:memories of a survivor


In [29]:
logwarn('talking to a hierarchically upper logger', logger='libro')

In [30]:
logerror('talking to an unrelated logger', logger='rivista.cucina')

Notice that loggers are automatically defined by the simple action of naming them.  What happened above is that the handler we created, attached to the `libro.romanzo` logger, only saw the records going to the loggers below its logger.  All record going to hierarchically upper loggers or to unrelated loggers are not logged, regardless of their severity.

Also notice that the text printed doesn't contain any more that "::" between the two ":" there's the name of the logger that received the logging record in the first place.

## Logger Objects

In the last example box in the previous section we have sent logging records to the "libro.romanzo.companile" logger.  We have done this by invoking the global loginfo function, passing the name of the logger.  This is only practical if you are logging to the root logger or if you are using many differnt loggers, not if you are sending, as in our example, more records to the same logger.  If you are taling the whole time to the same logger, you do not want to have to repeat the name of the logger each time you send it a record.

The solution to this is in the object oriented approach taken in this logging library.  The `getLogger()` function returns a Logger object, which, since we are using Reference Classes, is itself an environment.  In the previous examples we have only used the fact that Logger objects are environments, let's now have a look at what more they offer.

In [31]:
str(getLogger())

Reference class 'Logger' [package "logging"] with 4 fields
 $ name        : chr ""
 $ handlers    : Named list()
 $ level       : Named num 20
  ..- attr(*, "names")= chr "INFO"
 $ msg_composer:function ()  
 and 32 methods, of which 18 are  possibly relevant:
   .deducelevel, .logrecord, addHandler, debug, error, fine, finer, finest,
   getHandler, getLevel, getMsgComposer, getParent, info, log, removeHandler,
   setLevel, setMsgComposer, warn


In [32]:
is.environment(getLogger())

[1] TRUE

Let me keep it compact, I'm just giving you the code that will produce the same logging as in the previous example.  Do notice that you can mix invoking object methods with usage of global functions.

In [33]:
logReset()

In [34]:
getLogger('libro.romanzo')$addHandler(writeToConsole)

In [35]:
lrc <- getLogger('libro.romanzo.companile')

In [36]:
lrc$info('Ma cos\'è questo amore?')

2019-07-21 01:26:11 INFO:libro.romanzo.companile:Ma cos'è questo amore?


In [37]:
lrc$info('Se la luna mi porta fortuna')

2019-07-21 01:26:11 INFO:libro.romanzo.companile:Se la luna mi porta fortuna


In [38]:
lrc$info('Giovanotti, non esageriamo!')

2019-07-21 01:26:11 INFO:libro.romanzo.companile:Giovanotti, non esageriamo!


In [39]:
loginfo('memories of a survivor', logger='libro.romanzo.lessing')

2019-07-21 01:26:11 INFO:libro.romanzo.lessing:memories of a survivor


In [40]:
getLogger('libro')$warn('talking to a hierarchically upper logger')

In [41]:
logerror('talking to an unrelated logger', logger='rivista.cucina')

## Logging to File

Actually the name of this paragraph is misleading.  A more correct name would be *handling to file*, since it's a handler and not a logger that is actually writing some representation of your logrecords to a file. 

To make sure log records are sent to a file, you choose a logger and attach to a it a handler with a function that writes to your file.  The logging package exports the commodity function `writeToFile` for this purpose.  The name of the file is given as an extra parameter in the call to `addHandler`.

Recall that both loggers *and* handlers have a level.  Records at a specific severity are examined by loggers first; if the severity is higher than the level of the logger, they are offered to all of the attached handlers.  Handlers will again check the level of the record before taking action.  In the following example we make sure absolutely all logrecords are examined by initializing the root logger at the `FINEST` level.  The level of the `basic_stdout` console handler is not affected.

In [42]:
logReset()

In [43]:
basicConfig(level='FINEST')

In [44]:
addHandler(writeToFile, file='~/testing.log', level='DEBUG')

In [45]:
with(getLogger(), names(handlers))

[1] "basic.stdout" "writeToFile"

In [46]:
loginfo('test %d', 1)

2019-07-21 01:26:11 INFO::test 1


In [47]:
logdebug('test %d', 2)

2019-07-21 01:26:11 DEBUG::test 2


In [48]:
logwarn('test %d', 3)

2019-07-21 01:26:11 WARNING::test 3


In [49]:
logfinest('test %d', 4)

2019-07-21 01:26:11 FINEST::test 4


If the file was not existing or empty, this would be its content after the above steps:

2019-07-21 00:59:40 INFO::test 1 <br>
2019-07-21 00:59:51 DEBUG::test 2 <br>
2019-07-21 01:00:00 WARNING::test 3 <br>

All log records have been passed to both handlers `basic.stdout` and `writeToFile`.  The default console handler has handled records with severity at or above `INFO`, our file handler had threshold `DEBUG` so it handled also the second record in the example session.  The fourth  record was dropped by both handlers.

## Formatting Your Log Records

In this session we are going to see how to generate a diagnostics file for a system that organizes logrecords in a different way than Python.

In [50]:
formatter.fewsdiagnostics <- function(record) {
    if (record$level <= loglevels[['INFO']])
        level <- 3
    else if (record$level <= loglevels[['WARNING']])
        level <- 2
    else if (record$level <= loglevels[['ERROR']])
        level <- 1
    else
        level <- 0
    
    sprintf('  <line level="%d" description="LizardScripter :: %s :: %s"/>\n', level, 
            record$timestamp, record$msg)
}

Notice that the field `$msg` of a record is already "formatted", as we have seen with `logwarn('my %s is %d', 'name', 5)`.  That part can be used but not undone any more.

When you add a handler to a logger, you can use the `formatter` parameter to associate to the handler a function that takes a logrecord and returns a string.  The above example function is such a function.

The formatter you can associate to a handler can combine the tags in the logrecord to produce a string.  The tags that are available in a logrecord are: `$logger` (the name of the logger which produced the record), `$msg`, `$timestamp`, `$level` (numeric), `$levelname` (character).

If you don't specify the `formatter` parameter, the default formatter is used, which looks like this:

In [51]:
defaultFormat <- function(record) {
    text <- paste(record$timestamp, paste(record$levelname, record$logger, record$msg, sep=":"))
}

The rest of the code, just slightly simplified, showing how we actually use this capability is given here.

Notice that the `diagonostics` handler we add will not handle `DEBUG` logrecords.

In [52]:
setup.fewsdiagnostics <- function(filename) {
    cat('<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n', file=filename, append=FALSE)
    cat('<Diag version="1.2" xmlns="..." xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xsi:schemaLocation="...">\n', file=filename, append=FALSE)
    addHandler('diagnostics', 
              writeToFile, file=filename,
              logger='fews.diagnostics',
              formatter=formatter.fewsdiagnostics)
}

teardown.fewsdiagnostics <- function(filename) {
    cat('</Diag>\n', file=filename, append=TRUE)
    removeHandler('diagnostics', logger='fews.diagnostics')
}

## Writing Your Own Handlers

Different from in the logging library in Python and in Java, handlers in this logging library aren't objects: they are environments stored in one of the loggers.  The principle characteristic property of a handler is its `action`.  An action is a function that specifies what the handler should do with a logrecord that, based on all that we have seen above, must be handled.  The to commodity functions we have seen in the first two sessions, `writeToConsole` and `writeToFile` are action functions.

A look at `writeToFile` will help understand the idea implemented in this library:

In [53]:
writeToFile

function (msg, handler, ...) 
{
    if (length(list(...)) && "dry" %in% names(list(...))) 
        return(exists("file", envir = handler))
    cat(paste0(msg, "\n"), file = with(handler, file), append = TRUE)
}
<bytecode: 0x7ff39ac777d0>
<environment: namespace:logging>

An action is invoked if a record must be handled.  Its result code is ignored and all its output goes to the console.  It receives exactly two arguments, the formatted message that must be output (the string returned by the formatter of the handler) and the handler owning the action.  Recall that a handler is an environment: in the action you can inspect the `handler` environment to perform the desired behaviour.

Imagine you want a handler to send its messages to an xmlrpc server or to a password protected ftp server, you would add these properties in the call to `addHandler`.  `addHandler` would store them in the new handler environment.  Your action function would retrieve the values from the handler and use them to connect to your hypothetical external server.

The structure of your solution might be something like this:

In [54]:
sendToFtpServer <- function(msg, handler) {
    proxy <- connectToServer(with(handler, server), with(handler, user), with(handler, passwd))
    do_the_rest()
}

addHandler(sendToFtpServer, user='', server='', passwd='', logger='deep.deeper.deepest')

ERROR: Error in with(handler_env, action)(NA, handler_env, dry = TRUE): unused argument (dry = TRUE)
